In [1]:
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, TrainingArguments, Trainer
import evaluate
import numpy as np
import wandb
import datasets
from ray import tune
import ray
import os
from ray.air import session
from sklearn.metrics import accuracy_score, f1_score
import torch

In [2]:
save_path = './dataset'
dataset = load_dataset('abisee/cnn_dailymail', '3.0.0', cache_dir=save_path)

Using the latest cached version of the dataset since abisee/cnn_dailymail couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration '3.0.0' at dataset/abisee___cnn_dailymail/3.0.0/0.0.0/96df5e686bee6baa90b8bee7c28b81fa3fa6223d (last modified on Tue Apr 15 09:04:36 2025).


In [3]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name, cache_dir='./model')
model = BartForConditionalGeneration.from_pretrained(model_name, cache_dir='./model')

In [4]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value



In [5]:
# 冻结编码器部分的所有层
for param in model.model.encoder.parameters():
    param.requires_grad = False

# 仅训练解码器部分的层
for param in model.model.decoder.parameters():
    param.requires_grad = True

In [6]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["highlights"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["highlights"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    
    # 显式保留原始文本列
    model_inputs["article"] = examples["article"]
    model_inputs["highlights"] = examples["highlights"]
    return model_inputs

# 应用预处理函数时，仅移除不需要的列（如果有）
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=[]  # 移除原始数据集中其他不需要的列（如果有）
    # 如果不需要移除任何列，可以设置为 remove_columns=[]
)

In [7]:
# tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])

In [8]:
# tokenized_datasets.save_to_disk("./dataset/tokenized_datasets_2")
# tokenized_datasets = datasets.load_from_disk("./dataset/tokenized_datasets")

In [9]:
# import evaluate
def compute_metrics(pred):
    try:
        rouge = evaluate.load('rouge')
        
        pred_ids = pred.predictions
        label_ids = pred.label_ids
        
        # 如果predictions是一个包含logits的数组，取argmax
        if len(pred_ids.shape) == 3:
            pred_ids = np.argmax(pred_ids, axis=-1)
        
        # 处理-100（特殊填充值）
        label_ids[label_ids == -100] = tokenizer.pad_token_id
        
        # 解码为文本
        pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
        
        # 计算ROUGE分数
        result = rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True)
        
        # 只返回中间F1分数
        return {
            "rouge1": result["rouge1"].mid.fmeasure,
            "rouge2": result["rouge2"].mid.fmeasure,
            "rougeL": result["rougeL"].mid.fmeasure
        }
    except Exception as e:
        print(f"计算指标时出错：{e}")
        # 返回默认值以避免训练中断
        return {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}

In [10]:
# training_args = TrainingArguments(
#     run_name = "Epoch_1_test",
#     output_dir="./results",
#     eval_strategy="epoch",
#     save_total_limit=3,
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     gradient_accumulation_steps=4,
#     num_train_epochs=1,
#     weight_decay=0.01,
#     logging_steps=100,
#     fp16=True,
# )

In [11]:
train_size = 1000  # You can adjust this number to use fewer examples
eval_size = 200  # Same for evaluation dataset

train_subset = tokenized_datasets["train"].select(range(train_size))
eval_subset = tokenized_datasets["validation"].select(range(eval_size))

# trainer = Trainer(
#     model=model,  # Your pre-trained model
#     args=training_args,  # Pass the training arguments
#     # train_dataset=tokenized_datasets["train"], 
#     # eval_dataset=tokenized_datasets["validation"],
#     train_dataset=train_subset,
#     eval_dataset=eval_subset,
#     tokenizer=tokenizer,  # Pass the tokenizer
#     # compute_metrics=compute_metrics,
# )
# wandb.init(project="Mlops-summary", entity="yunchiz-new-york-university")
# trainer.train()
# wandb.finish()

In [12]:
def train_fn(config, model, train_dataset, eval_dataset):
    try:
        trial_dir = session.get_trial_dir()  # 例如：~/ray_results/test/trial_xxx/
        output_dir = os.path.join(trial_dir, "results")
    except Exception as e:
        print(f"路径错误: {str(e)}")
        raise


    training_args = TrainingArguments(
        run_name = "ray_test_epoch_2",
        output_dir=output_dir,
        num_train_epochs=2,  
        
        # per_device_train_batch_size=config["batch_size"],  # Hyperparameter from Ray Tune
        # per_device_eval_batch_size=config["batch_size"],   # Hyperparameter from Ray Tune
        # gradient_accumulation_steps=config["gradient_accumulation_steps"],               # Hyperparameter from Ray Tune

        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        
        learning_rate=config["learning_rate"],              # Hyperparameter from Ray Tune
        
        weight_decay=0.01,
        logging_dir=os.path.join(trial_dir, "logs"),  
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",

        save_total_limit=3,
        metric_for_best_model="rougeL",
        fp16=True,
    )

    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=train_subset, 
        eval_dataset=eval_subset, 
        compute_metrics=compute_metrics,
    )
    try:
        # Train the model
        trainer.train()
    except Exception as e:
        print(f"训练失败: {str(e)}")
        raise

    try:
    # Evaluate the model
        eval_results = trainer.evaluate()
    except Exception as e:
        print(f"评估失败: {str(e)}")
        raise

    try:
    # Return the evaluation results to Ray Tune
        tune.report(metrics=eval_results)
        trainer.save_model(output_dir)
        tune.report(
            metrics=eval_results,
            checkpoint=tune.Checkpoint.from_directory(output_dir)  # 将模型目录作为检查点
        )
    except Exception as e:
        print(f"报告错误: {str(e)}")
        raise

    

In [13]:
search_space = {
    # "learning_rate": tune.grid_search([1e-5, 2e-5, 5e-5]),
    # "batch_size": tune.choice([8, 16]),
    # "warmup_steps": tune.choice([500, 1000, 2000]),
    "learning_rate": tune.grid_search([1e-5]),
}

In [14]:
wandb.init(project="Mlops-summary", entity="yunchiz-new-york-university")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yunchiz (yunchiz-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [15]:
current_dir = os.getcwd()
storage_path = f"file://{current_dir}/ray_results"

train_fn_with_params = tune.with_parameters(train_fn, model=model, train_dataset=train_subset, eval_dataset=eval_subset)
ray.init(ignore_reinit_error=True)  # Initialize Ray
analysis = tune.run(
    train_fn_with_params,  # The training function that Ray Tune will use
    config=search_space,  # The search space of hyperparameters
    # resources_per_trial={"cpu": 1, "gpu": 1},
    resources_per_trial={"cpu": 0, "gpu": 1},
    num_samples=1,  # Number of trials (hyperparameter combinations)
    verbose=1,  # Verbosity level of Ray Tune
    storage_path=storage_path,
    name="ray_test_epoch_2",
)

(train_fn pid=15050) wandb: Currently logged in as: yunchiz (yunchiz-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(train_fn pid=15050) wandb: Tracking run with wandb version 0.19.9
(train_fn pid=15050) wandb: Run data is saved locally in /tmp/ray/session_2025-04-15_10-47-35_553542_13907/artifacts/2025-04-15_10-47-48/ray_test_epoch_2/working_dirs/train_fn_04641_00000_0_learning_rate=0.0000_2025-04-15_10-47-48/wandb/run-20250415_104805-q97xhpey
(train_fn pid=15050) wandb: Run `wandb offline` to turn off syncing.
(train_fn pid=15050) wandb: Syncing run ray_test_epoch_2
(train_fn pid=15050) wandb: ⭐️ View project at https://wandb.ai/yunchiz-new-york-university/huggingface
(train_fn pid=15050) wandb: 🚀 View run at https://wandb.ai/yunchiz-new-york-university/huggingface/runs/q97xhpey
 52%|█████▏    | 32/62 [00:20<00:14,  2.05it/s]
(train_fn pid=15050) 
  0%|          | 0/25 [00:00<?, ?it/s]
(train_fn pid=15050) 
 12%|█▏        | 3/25 [00:00<00:0

(train_fn pid=15050) 计算指标时出错：'tuple' object has no attribute 'shape'
(train_fn pid=15050) {'eval_loss': 1.9222586154937744, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_runtime': 11.9726, 'eval_samples_per_second': 16.705, 'eval_steps_per_second': 2.088, 'epoch': 1.0}


                                               
100%|██████████| 25/25 [00:11<00:00,  5.76it/s]
                                               /root/miniconda3/lib/python3.12/site-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
(train_fn pid=15050)   warnings.warn(
  0%|          | 0/25 [00:00<?, ?it/s]
(train_fn pid=15050) 
 12%|█▏        | 3/25 [00:00<00:00, 22.54it/s]
(train_fn pid=15050) 
 24%|██▍       | 6/25 [00:00<00:01, 17.59it/s]
(train_fn pid=15050) 
 32%|███▏      | 8/25 [00:00<00:01, 16.41it/s]
(train_fn pid=15050) 
 40%|████      | 10/25 [00:00<00:00, 15.79it/s]
(train_fn pid=15050) 
 48%|████▊     | 12/25 [00:0

(train_fn pid=15050) 计算指标时出错：'tuple' object has no attribute 'shape'
(train_fn pid=15050) {'eval_loss': 1.3202884197235107, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_runtime': 10.3793, 'eval_samples_per_second': 19.269, 'eval_steps_per_second': 2.409, 'epoch': 1.96}


                                               
100%|██████████| 25/25 [00:10<00:00, 14.00it/s]
                                               


(train_fn pid=15050) {'train_runtime': 82.2217, 'train_samples_per_second': 24.324, 'train_steps_per_second': 0.754, 'train_loss': 3.1087245325888357, 'epoch': 1.96}


100%|██████████| 25/25 [00:01<00:00, 14.29it/s]


(train_fn pid=15050) 计算指标时出错：'tuple' object has no attribute 'shape'


100%|██████████| 25/25 [00:09<00:00,  2.57it/s]
2025-04-15 10:50:10,267	WARNING experiment_state.py:233 -- Saving the experiment state (which holds a global view of trial statuses and is used to restore the experiment) has already taken 30.24 seconds, which may cause consistency issues upon restoration if your driver script ungracefully exits.
This could be due to a large number of trials, large logfiles from lots of reported metrics, or throttling from the remote storage if uploading too frequently.
You may want to consider switching the `RunConfig(storage_filesystem)` to a more performant storage backend such as s3fs for a S3 storage path.
You can suppress this error by setting the environment variable TUNE_WARN_SLOW_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a higher value than the current threshold (30.0).
(train_fn pid=15050) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/autodl-tmp/ray_results/ray_test_epoch_2/train_fn_04641_00000_0_learning_rate=0.000

In [16]:
test_size = 200  # Same for evaluation dataset
test_dataset = tokenized_datasets["test"].select(range(test_size))

best_trial = analysis.get_best_trial(metric="eval_accuracy", mode="max")

# 获取检查点路径（通过 checkpoint 属性）
best_checkpoint = best_trial.checkpoint
best_checkpoint_dir = best_checkpoint.to_directory()  # 提取检查点目录
print(f"最佳模型路径：{best_checkpoint_dir}")

# 加载模型
from transformers import AutoModel
best_model = BartForConditionalGeneration.from_pretrained(best_checkpoint_dir)

最佳模型路径：/tmp/checkpoint_tmp_f69b548edfae4ccb947b243d69a34093


/root/miniconda3/lib/python3.12/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [17]:
# trainer = Trainer(
#     model=best_model,
#     args=TrainingArguments(output_dir="./tmp"),  # 临时目录，仅用于预测
# )

# predictions = trainer.predict(test_dataset)
# predictions_logits = predictions.predictions
# # predicted_labels = np.argmax(predictions_logits, axis=1)

# accuracy = accuracy_score(test_dataset["labels"], predictions_logits)
# f1 = f1_score(test_dataset["labels"], predictions_logits, average="macro")

# results = rouge.compute(predictions=predictions, references=labels)

# print("results")

In [18]:
gen_kwargs = {
    "max_length": 128,          # 生成摘要的最大长度
    "min_length": 30,           # 生成摘要的最小长度
    "num_beams": 4,             # Beam Search 的 beam 数
    "length_penalty": 2.0,      # 长度惩罚系数（>1鼓励更长，<1鼓励更短）
    "no_repeat_ngram_size": 3,  # 禁止重复的 n-gram 大小
    "early_stopping": True,     # 是否提前停止生成
}

In [19]:
def evaluate_testset(test_data, model, tokenizer):
    """
    评估测试集并返回 ROUGE 指标
    Args:
        test_data (Dataset): 预处理后的测试数据集（需包含 "article" 和 "highlights"）
        model (PreTrainedModel): 加载的最佳模型
        tokenizer (PreTrainedTokenizer): 对应的 tokenizer
    Returns:
        dict: ROUGE 指标结果
    """
    try:
        # 将模型移动到 GPU（如果可用）
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        
        # 存储预测和真实摘要
        pred_summaries = []
        true_summaries = []
        
        # 批量生成预测
        batch_size = 8  # 根据 GPU 显存调整
        for i in range(0, len(test_data), batch_size):
            batch = test_data.select(range(i, min(i+batch_size, len(test_data))))
            
            # 编码输入文本
            inputs = tokenizer(
                batch["article"],
                max_length=1024,
                truncation=True,
                padding="max_length",
                return_tensors="pt"
            ).to(device)
            
            # 生成摘要
            with torch.no_grad():
                summaries = model.generate(
                    input_ids=inputs["input_ids"],
                    attention_mask=inputs["attention_mask"],
                    **gen_kwargs
                )
            
            # 解码预测和真实摘要
            decoded_preds = tokenizer.batch_decode(summaries, skip_special_tokens=True)
            decoded_labels = [highlight for highlight in batch["highlights"]]
            
            pred_summaries.extend(decoded_preds)
            true_summaries.extend(decoded_labels)
            
            print(f"Processed {i + batch_size}/{len(test_data)} samples")
            
        # 计算 ROUGE 指标
        rouge = evaluate.load("rouge")
        results = rouge.compute(
            predictions=pred_summaries,
            references=true_summaries,
            use_stemmer=True
        )
        
        # 格式化结果（保留4位小数）
        return {
            "rouge1": round(results["rouge1"], 4),
            "rouge2": round(results["rouge2"], 4),
            "rougeL": round(results["rougeL"], 4),
        }
    
    except Exception as e:
        print(f"评估过程中发生错误: {str(e)}")
        raise


In [20]:
try:
    # 加载测试数据集（假设已预处理）
    # test_size = 200
    # test_dataset = tokenized_datasets["test"].select(range(test_size))
    
    # # 加载最佳模型检查点
    # best_trial = analysis.get_best_trial(metric="metrics", mode="max")  # 请根据实际指标名称修改
    # best_checkpoint_dir = best_trial.checkpoint.to_directory()
    
    # # 加载模型和 tokenizer
    # tokenizer = BartTokenizer.from_pretrained(best_checkpoint_dir)
    # model = BartForConditionalGeneration.from_pretrained(best_checkpoint_dir)
    
    # 执行评估
    results = evaluate_testset(test_dataset, best_model, tokenizer)
    
    # 打印结果
    print("\n测试集评估结果:")
    print(f"ROUGE-1: {results['rouge1']}")
    print(f"ROUGE-2: {results['rouge2']}")
    print(f"ROUGE-L: {results['rougeL']}")

except KeyError as e:
    print(f"数据加载错误: 请检查数据集是否包含 'article' 和 'highlights' 字段 - {str(e)}")
except FileNotFoundError as e:
    print(f"模型加载错误: 检查点路径 {best_checkpoint_dir} 不存在 - {str(e)}")
except Exception as e:
    print(f"未知错误: {str(e)}")

/root/miniconda3/lib/python3.12/site-packages/transformers/generation/utils.py:1667: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Processed 8/200 samples
Processed 16/200 samples
Processed 24/200 samples
Processed 32/200 samples
Processed 40/200 samples
Processed 48/200 samples
Processed 56/200 samples
Processed 64/200 samples
Processed 72/200 samples
Processed 80/200 samples
Processed 88/200 samples
Processed 96/200 samples
Processed 104/200 samples
Processed 112/200 samples
Processed 120/200 samples
Processed 128/200 samples
Processed 136/200 samples
Processed 144/200 samples
Processed 152/200 samples
Processed 160/200 samples
Processed 168/200 samples
Processed 176/200 samples
Processed 184/200 samples
Processed 192/200 samples
Processed 200/200 samples

测试集评估结果:
ROUGE-1: 0.3567
ROUGE-2: 0.1528
ROUGE-L: 0.263


In [21]:
wandb.finish()

In [22]:
# def summarize(text):
#     inputs = tokenizer([text], max_length=1024, truncation=True, return_tensors="pt")
#     summary_ids = best_model.generate(inputs["input_ids"], max_length=128, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# print(summarize("""New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
# A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
# Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
# In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
# Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
# 2010 marriage license application, according to court documents.
# Prosecutors said the marriages were part of an immigration scam.
# On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
# After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
# Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
# All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
# Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
# Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
# The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
# Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
# Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
# If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18."""))

Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. If convicted, she faces up to four years in prison.